In [78]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

In [79]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Dataset") \
    .getOrCreate()

In [80]:
path = "../ratings.csv"
df = spark.read.csv(path, header=True)
df.show(5)

Py4JJavaError: An error occurred while calling o399.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:

In [ ]:
path2 = "../movies.csv"
df2 = spark.read.csv(path2, header=True)
df2 = df2.withColumnRenamed("movieId", "movieId_movies")
df2.show(5)

+--------------+--------------------+--------------------+
|movieId_movies|               title|              genres|
+--------------+--------------------+--------------------+
|             1|    Toy Story (1995)|Adventure|Animati...|
|             2|      Jumanji (1995)|Adventure|Childre...|
|             3|Grumpier Old Men ...|      Comedy|Romance|
|             4|Waiting to Exhale...|Comedy|Drama|Romance|
|             5|Father of the Bri...|              Comedy|
+--------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
df = df.join(df2, df.movieId == df2.movieId_movies, 'left')
df.show(truncate=False)

+------+-------+------+----------+--------------+------------------------------------------------------------------+-------------------------------+
|userId|movieId|rating|timestamp |movieId_movies|title                                                             |genres                         |
+------+-------+------+----------+--------------+------------------------------------------------------------------+-------------------------------+
|1     |296    |5.0   |1147880044|296           |Pulp Fiction (1994)                                               |Comedy|Crime|Drama|Thriller    |
|1     |306    |3.5   |1147868817|306           |Three Colors: Red (Trois couleurs: Rouge) (1994)                  |Drama                          |
|1     |307    |5.0   |1147868828|307           |Three Colors: Blue (Trois couleurs: Bleu) (1993)                  |Drama                          |
|1     |665    |5.0   |1147878820|665           |Underground (1995)                                       

In [ ]:
df = df.drop('movieId_movies')

In [ ]:
df = df.withColumn("genres", split(df.genres, r"\|"))

In [ ]:
df = df.withColumn("genres", explode(df.genres).alias("genre"))
df.show()

+------+-------+------+----------+--------------------+--------+
|userId|movieId|rating| timestamp|               title|  genres|
+------+-------+------+----------+--------------------+--------+
|     1|    296|   5.0|1147880044| Pulp Fiction (1994)|  Comedy|
|     1|    296|   5.0|1147880044| Pulp Fiction (1994)|   Crime|
|     1|    296|   5.0|1147880044| Pulp Fiction (1994)|   Drama|
|     1|    296|   5.0|1147880044| Pulp Fiction (1994)|Thriller|
|     1|    306|   3.5|1147868817|Three Colors: Red...|   Drama|
|     1|    307|   5.0|1147868828|Three Colors: Blu...|   Drama|
|     1|    665|   5.0|1147878820|  Underground (1995)|  Comedy|
|     1|    665|   5.0|1147878820|  Underground (1995)|   Drama|
|     1|    665|   5.0|1147878820|  Underground (1995)|     War|
|     1|    899|   3.5|1147868510|Singin' in the Ra...|  Comedy|
|     1|    899|   3.5|1147868510|Singin' in the Ra...| Musical|
|     1|    899|   3.5|1147868510|Singin' in the Ra...| Romance|
|     1|   1088|   4.0|11

In [ ]:
df.filter(df.rating.isNull()).show(10)

+------+-------+------+---------+-----+------+
|userId|movieId|rating|timestamp|title|genres|
+------+-------+------+---------+-----+------+
+------+-------+------+---------+-----+------+



In [ ]:
df = df.select("userId", "movieId", "rating", "title", "genres")
df.show(5)

+------+-------+------+--------------------+--------+
|userId|movieId|rating|               title|  genres|
+------+-------+------+--------------------+--------+
|     1|    296|   5.0| Pulp Fiction (1994)|  Comedy|
|     1|    296|   5.0| Pulp Fiction (1994)|   Crime|
|     1|    296|   5.0| Pulp Fiction (1994)|   Drama|
|     1|    296|   5.0| Pulp Fiction (1994)|Thriller|
|     1|    306|   3.5|Three Colors: Red...|   Drama|
+------+-------+------+--------------------+--------+
only showing top 5 rows



In [ ]:
# Verificando a contagem de linhas
print(f"Contagem de linhas antes de salvar: {df.count()}")

# Verificando o esquema do DataFrame
df.printSchema()

Contagem de linhas antes de salvar: 67809886
root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = false)



In [ ]:
# Salvando o resultado em um CSV
df.write.csv("movies_ratings.csv", header=True, mode='overwrite')

Py4JJavaError: An error occurred while calling o386.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java: